# Data Mining
***
## | Mining Frequent Itemsets from Transactions |

Damien Draime - May 19

### Introduction

Given a **database of transactions**, it is often interesting to know what itemsets are frequent, assuming a certain support threshold. Once those itemsets are found, one can derive **association rules** which indicate what itemsets often co-occur in transactions. For instance, if the database is made of sales transactions of a supermarket, a association rule that could emerge from the dataset is \[Beer, Charcoal, Sausages\] $\rightarrow$ \[Matches\]. This kind of association rules can be useful for the store owner in order to arrange the store layout, for example. <br> However, mining frequent itemsets is not as straightforward and requires tailored algorithms. Two of the best known ones will be covered in this notebook, namely: **Apriori, and PCY algorithms**. Some references will be given for some other algorithms. We also expect the reader to be familiar with the notions of support, confidence, itemset,  frequent itemset, etc..<br> 

<br> *Note that transaction / basket often refers to sales transaction but we could apply algorithms covered in this notebook to other types of transaction. For example, a transaction could represent a browser session and items in this transaction would be all websites visited during that session. It could then be useful to know which websites are often co-visited in order to preload a website if the rule body is satisfied.*<br>
<br>
 - [Motivation](#Motivation)
 - [Apriori](#Apriori)
     - [Observation](#Observation)
     - [Implementation](#Implementation)
     - [Limitations](#Limitations)
 - [PCY](#Training)
     - [Observation](#Observation)
     - [Implementation](#Implementation)
     - [Limitations](#Limitations)
 - [Other approaches](#Other-approaches)
     - [Multihash algorithm](#Multihash-algorithm)
     - [Toivonen's algorithm](#Toivonen's-algorithm)
     - [FP-Tree](#FP-Tree)
 - [Conclusion](#Conclusion)


***

*Based on __[Agrawal et al. - Mining Association Rules between Sets of Items in Large Databases](https://rakesh.agrawal-family.com/papers/sigmod93assoc.pdf)__, __[Park, Chen, and Yu - An Effective Hash-Based Algorithm for Mining Association Rules](https://www.researchgate.net/publication/2267911_An_Effective_Hash-Based_Algorithm_for_Mining_Association_Rules)__ and __[KU Leuven's Data Mining - Lectures 07-08](https://p.cygnus.cc.kuleuven.be/bbcswebdav/pid-24321835-dt-content-rid-212707824_2/courses/B-KUL-H02C6a-1819/jdavis-assoc-rule-part1%281%29.pdf)__.*<br>
*Other source: __[Mining of Massive Datasets](http://www.mmds.org/)__*

### Motivation
At first sight, we might be tempted to simply use a **generate and test algorithm** to solve this task of finding frequent itemsets from a database of transactions. We would then simply generate all possible itemsets of different sizes and test if they have a support greater than the support threshold. However, if we take again the examples mentioned above, a retailer such as WalMart would have around 100k items. And what about the number of websites... . Generating the candidate itemsets is a combinatorial problem: we would have to generate $C_2(100,000) = {100,000 \choose 2} = 4,999,950,000$ pairs, in the case of WalMart. Storing all those candidate pairs in memory would be complicated. Not mentioning that multiple passes would be necessary to test the different candidate itemset sizes, but the transaction database's size of companies such as WalMart can be in the order of billions of transactions. Clearly we need to have tailored algorithms for this task. <br>
What we discovered so far is that:
- generating *candidate pairs* will be the bottleneck of our naive algorithm since it is a combinatorial problem;
- we would like to store candidates in memory to limit the disk I/O;
- storing the database in memory is not possible.

The toy database we will use in this notebook is the following:

In [2]:
database_file = 'dataset\Transactions_db.txt'
with open(database_file) as database:
        for i, transaction in enumerate(database):
            print('Transaction #{}: {}'.format(i,transaction))

Transaction #0: 1,4,10

Transaction #1: 3,5,6,3

Transaction #2: 3,5,6,8

Transaction #3: 3,4,6

Transaction #4: 3,5,6,8

Transaction #5: 2,6,7,8

Transaction #6: 2,6,7,8

Transaction #7: 1,4,9

Transaction #8: 3,4

Transaction #9: 3,5,6,7



As you can see, we will limit ourselves to 10 transactions. Again, you can be creative with the items in a transaction. Items, represented by numbers here, can be items bought by a customer, websites visited in a browser session, words used in a query, ....<br>
The first algorithm we will turn ourselves to is the Apriori algorithm.

### Apriori Algorithm
#### Observation
The **Apriori algorithm** is built around the observation that *an itemset can only be frequent if all its subsets are also frequent*. This simple observation is extremely powerful because it implies that, given the supports of all the subsets of a candidate itemset then we can know for sure when the latter has a chance to be frequent. For instance, the candidate itemset $$[3, 4, 6]$$ can only be frequent if $$[3], [4], [6], [3, 4], [3, 6], [4, 6]$$ are also frequent. But note that if we use recursion, we only need to know the status of the subsets of size $s-1$, where $s$ is the size of the candidate itemset. Hence, we are only interested in the support of \[3, 4\], \[3, 6\], \[4, 6\]. Indeed, if those are frequent then it means that \[3\], \[4\], \[6\] were also frequent.<br><br>
This **property** is extremely useful because it suggests that we won't have to generate all candidates. For example, for the itemsets of size $s=2$, we should only generate, and latter test, candidates for which the two items are also frequent. Similarly, a candidate of size $s=3$ needs only to be generated if its building pairs are also frequent.
#### Implementation
The Apriori will exploit this notion by **passing through the entire database multiple times**, one time for testing candidates of a specific size $s$ it has generated. Thus, the first pass through the database is used to identify frequent itemsets of size $s=1$, the second for all generated pairs, etc.. Between two passes, the algorithm will generate a set of relevant candidates of size $s+1$ based on the information collected during the last pass which was about itemsets of size $s$. The algorithm will thus end when the set of relevant candidates is empty.<br>
The generation of the candidates itemsets for the next pass is performed in two steps:
- Join
- Prune

The **Join Step** is used to generate of first draft of the set of candidates. Based on the list of frequent itemsets of size $s-1$ identified during the last pass, it will repetitively look for two itemsets that share the same first items but not the last one and join them to form a candidate itemset of size $s$.<br>
For example, if these are the frequent itemsets identified during the last pass:
$${[1, 2, 4], [1, 3, 4], [1, 3, 5], [2, 5, 6], [2, 5, 7], [2, 5, 9], [2, 7, 9], [5, 7, 9], [7, 9, 10]}$$
then the first draft of the new set of candidates generated by the Join step would be:
$${[1, 3, 4, 5], [2, 5, 6, 7], [2, 5, 6, 9], [2, 5, 7, 9]}$$
This can easily be done with two *for* loops:

In [1]:
freq_itemsets = [[1, 2, 4], [1, 3, 4], [1, 3, 5], [2, 5, 6], [2, 5, 7], [2, 5, 9], [2, 7, 9], [5, 7, 9], [7, 9, 10]]
for i, itemseti in enumerate(freq_itemsets[:-1]):
    for itemsetj in freq_itemsets[i+1:]:
        *itemseti_first, itemseti_last = itemseti
        *itemsetj_first, itemsetj_last = itemsetj
        if itemseti_first == itemsetj_first:
            print (itemseti_first + [itemseti_last] + [itemsetj_last])

[1, 3, 4, 5]
[2, 5, 6, 7]
[2, 5, 6, 9]
[2, 5, 7, 9]


However, there is a problem with this set of candidates. It generates more candidates than it should because it doesn't respect the Apriori principle. Indeed, have you noticed that the actually only the last candidate has all its subsets of size $s-1$ frequent ? The candidate $[1, 3, 4, 5]$ should be rejected because $[3, 4, 5]$. Similarly, $[2, 5, 6, 7]$, and $[2, 5, 6, 9]$ have to be removed from the set because one of their subsets is not frequent. This is why this first set is a draft that will be submitted to the Prune step.

In [3]:
def join(freq_itemsets):
    k = len(freq_itemsets[0]) + 1
    new_candidates = []
    if k == 2:
        for i, itemseti in enumerate(freq_itemsets[:-1]):
            for itemsetj in freq_itemsets[i+1:]:
                new_candidates.append(tuple([itemseti] + [itemsetj]))
    else:
        for i, itemseti in enumerate(freq_itemsets[:-1]):
            for itemsetj in freq_itemsets[i+1:]:
                *itemseti_first, itemseti_last = itemseti
                *itemsetj_first, itemsetj_last = itemsetj
                if itemseti_first == itemsetj_first:
                    new_candidates.append(tuple(itemseti_first + [itemseti_last] + [itemsetj_last]))
    
    return new_candidates

The **Prune Step** is in charge of reviewing the first draft of candidates generated by the Join Step. For this we simply need to check all the subsets that we haven't tested in the Join Step. For example:

In [3]:
candidate = [1, 3, 4, 5]
for i in range(len(candidate) - 2):
    subset = candidate[:i] + candidate[i + 1 :]
    print(subset)

[3, 4, 5]
[1, 4, 5]


Those are indeed the two subsets we haven't tested yet. We can thus write the Prune Step:

In [4]:
def prune(candidates, freq_itemsets):
    kept_candidates = []
    for candidate in candidates:
        for i in range(len(candidate) - 2):
            subset = candidate[:i] + candidate[i + 1 :]
            if subset not in freq_itemsets:
                break
        else:
            kept_candidates.append(candidate)
    return kept_candidates

Note that is not necessary to apply the Prune Step when considering the candidates of size $s=2$ since all the subsets have already been checked during the Join Step. We now have everything we need to write the high level Apriori algorithm:

In [5]:
def first_pass(database_file):
    raw_count = {}
    with open(database_file) as file:
        for line in file:
            transaction = set(line[:-1].split(','))  # Creating a set to avoid counting multiple times a duplicated item from a transaction
            for itemset in transaction:
                raw_count[itemset] = raw_count.get(itemset, 0) + 1
    return raw_count

In [6]:
def apriori(database_file, support_threshold):
    raw_count = first_pass(database_file)
    candidates = raw_count.keys()
    vault_freq_itemsets = []
    fist_pass_check = True
    
    while len(candidates) > 0:
        freq_itemsets = []
        
        if not fist_pass_check:
            with open(database_file) as database:  # We can't store the database in memory
                for transaction in database:
                    transaction = list(set(transaction[:-1].split(',')))  # To eliminate duplicates that would trump the support count
                    for candidate in candidates:
                        if set(candidate).issubset(transaction):
                            raw_count[candidate] = raw_count.get(candidate, 0) + 1
        
        freq_itemsets = [itemset for itemset, support in raw_count.items() if support >= support_threshold]
        vault_freq_itemsets.append(freq_itemsets)
        
        candidates = join(freq_itemsets)
        
        if not fist_pass_check:
            candidates = prune(candidates, freq_itemsets)
        else:
            fist_pass_check = False
        
        raw_count = {}
        
    return vault_freq_itemsets

![Apriori algorithm illustration](img/Apriori.JPG)

In [7]:
print(apriori(database_file, 2))

[['1', '4', '6', '3', '5', '8', '2', '7'], [('1', '4'), ('6', '3'), ('6', '5'), ('3', '5'), ('6', '8'), ('3', '8'), ('5', '8'), ('4', '3'), ('6', '2'), ('6', '7'), ('8', '2'), ('8', '7'), ('2', '7')], [('6', '3', '5'), ('6', '3', '8'), ('6', '5', '8'), ('3', '5', '8'), ('6', '8', '2'), ('6', '8', '7'), ('6', '2', '7'), ('8', '2', '7')], [('6', '3', '5', '8'), ('6', '8', '2', '7')]]


In [11]:
frequent_itemsets = apriori(database_file, 2)
for size, frequent_itemset in enumerate(frequent_itemsets):
    print('Frequent itemsets of size {}:\n{}'.format(size+1, frequent_itemset),end='\n\n')

Frequent itemsets of size 1:
['1', '4', '6', '3', '5', '8', '2', '7']

Frequent itemsets of size 2:
[('1', '4'), ('6', '3'), ('6', '5'), ('3', '5'), ('6', '8'), ('3', '8'), ('5', '8'), ('4', '3'), ('6', '2'), ('6', '7'), ('8', '2'), ('8', '7'), ('2', '7')]

Frequent itemsets of size 3:
[('6', '3', '5'), ('6', '3', '8'), ('6', '5', '8'), ('3', '5', '8'), ('6', '8', '2'), ('6', '8', '7'), ('6', '2', '7'), ('8', '2', '7')]

Frequent itemsets of size 4:
[('6', '3', '5', '8'), ('6', '8', '2', '7')]



#### Limitations
As you can see the Apriori algorithm is way more efficient than the naive approach because it generates less candidates to test. However, multiple passes through the database are necessary which is not optimal. And particularly the **second pass is computationally intensive** because the set of candidates to test is the longest. This means that for each transaction, each candidate in the set need to be tested.... The second algorithm we will cover tries to address this problem specifically.

### PCY
The **Park-Chen-Yu algorithm**, also called PCY algorithm, addresses one of the limitations of the Apriori algorithm. It tries to limit the computationnal resources needed for the second pass through the database. This is done via hashing.
#### Observation
During the first pass of the Apriori algorithm through the database, most of the memory is idle. We could use the rest to help shorten the list of itemsets of size 2 that we will have to check during the second pass. We could achieve this by **hashing** pairs (i.e. itemsets of size 2) we encouter in each transaction to buckets. Each time a pair hashes to a bucket, we increment this bucket's count of 1. Note that it is not the same as counting occurrences of each pair for two reasons:
1. because there might be collisions (i.e. two different pairs hashing to the same buckets);
2. because we don't store which pair hashes to which bucket.

Then, given the "count" of each bucket, we can filter more aggressively the list of candidates of size 2 in the Pruning Step. Indeed, if a candidate pair does not hash to a bucket that has a "count" greater than, or equal to the support threshold, then this pair won't be frequent as well. However keep in mind that even with this additionnal filter, we still have candidates that are not frequent hence we still need to pass through the database once more to filter these out. But we will have to check less pairs in the second pass, which is the more computationally intensive one.<br>
Let's illustrate these last points:

In [14]:
def hashfunction(itemset, num_buckets):
    return sum([ord(element) for element in str(itemset)]) % num_buckets

In [28]:
pairs = [[1,4], [1,5], [2,3]]
for pair in pairs:
    print('The pair {} hashes to bucket #{}'.format(pair, hashfunction(pair, 3)))

The pair [1, 4] hashes to bucket #1
The pair [1, 5] hashes to bucket #2
The pair [2, 3] hashes to bucket #1


In this toy example, we only requested a hash function that created 3 buckets. It is then obvious that multiple pairs will hash to the same bucket and thus collide, as it is the case for pairs \[1, 4\], and \[2, 3\]. Hence if multiple pairs which individually are not frequent hash to the same bucket, inflating its "count" to the point where it's greater than the support threshold then we would have to check all those pairs in the second pass to discover that they are not frequent. Hence, it is important that the hash frequent reduce collisions as much as possible by evenly distributing pairs among the buckets. On the other hand, we don't want too many buckets because we don't want our hash table to take too much space in memory. Furthermore, to save space, this hash table will be saved as a bitmap. This is illustrated below:

In [31]:
def hashtable(candidates):
    hashtbl = {}
    num_buckets = 11  # Arbitrary choice, could be adjusted dynamically
    for candidate in candidates:
        hashvalue = hashfunction(candidate, num_buckets)
        hashtbl[hashvalue] = hashtbl.get(hashvalue, 0) + 1
    return hashtbl

In [32]:
def bitmap(hashtable, num_buckets, support_threshold):
    return [1 if hashtable.get(i, 0) >= support_threshold else 0 for i in range(num_buckets)]

In [33]:
def first_pass_and_hash(database_file, support_threshold):
    raw_count = {}
    pairs = []
    with open(database_file) as database:
        for transaction in database:
            transaction = set(transaction[:-1].split(','))  # Creating a set to avoid counting multiple times a duplicated item from a basket
            for itemset in transaction:
                raw_count[itemset] = raw_count.get(itemset, 0) + 1
            for i, itemseti in enumerate(list(transaction)[:-1]):
                for itemsetj in list(transaction)[i+1:]:
                    pairs.append(tuple([itemseti] + [itemsetj]))
            bitmap_hashtable = bitmap(hashtable(pairs), 11, support_threshold)

    return raw_count, bitmap_hashtable

In [37]:
raw_count, bitmap_hashtable = first_pass_and_hash(database_file, 2)
print('Raw count of itemsets of size 1:\n{}\n\nBitmap:\n{}'.format(raw_count, bitmap_hashtable))

Raw count of itemsets of size 1:
{'1': 2, '10': 1, '4': 4, '6': 7, '3': 6, '5': 4, '8': 4, '2': 2, '7': 3, '9': 1}

Bitmap:
[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1]


*Note that python as a hash function already implemented. The one used here is to showcase the fact that even if the hash function produces collisions, the algorithm will still work*

#### Implementation
We now have everything to implement this PCY algorithm. Just bear in mind that:
- this hashing trick is only used between the first and second passes but on future passes;
- the bitmap of the hash table is used in the Prune Step
- for a pair \[$i$, $j$\] to remain in the set of candidates for the second pass:
    - both $i$ and $j$ need to be frequent
    - the pair \[$i$, $j$\] needs to hash to a bucket with "count" $\geq$ support threshold

In [38]:
def prune(candidates, freq_itemsets, bitmap):
    kept_candidates = []
    for candidate in candidates:
        if bitmap is None or bitmap[hashfunction(candidate, 11)] == 1:
            for i in range(len(candidate) - 2):
                subset = candidate[:i] + candidate[i + 1 :]
                if subset not in freq_itemsets:
                    break
            else:
                kept_candidates.append(candidate)
    return kept_candidates

In [39]:
def pcy(database_file, support_threshold):
    raw_count, bitmap = first_pass_and_hash(database_file, support_threshold)
    first_pass = True
    candidates = raw_count.keys()
    vault_freq_itemsets = []
    
    while len(candidates) > 0:
        freq_itemsets = []
        
        if not first_pass:
            with open(database_file) as database:
                for transaction in database:
                    transaction = set(transaction[:-1].split(','))  # Creating a set to avoid counting multiple times a duplicated item from a basket
                    for candidate in candidates:
                        if set(candidate).issubset(transaction):
                            raw_count[candidate] = raw_count.get(candidate, 0) + 1
        
        freq_itemsets = [itemset for itemset, support in raw_count.items() if support >= support_threshold]
        vault_freq_itemsets.append(freq_itemsets)
                            
        candidates = join(freq_itemsets)
        
        candidates = prune(candidates, freq_itemsets, bitmap)
        
        if first_pass:
            first_pass = False
            bitmap = None
        raw_count = {}
        
    return vault_freq_itemsets

In [40]:
frequent_itemsets = pcy(database_file, 2)
for size, frequent_itemset in enumerate(frequent_itemsets):
    print('Frequent itemsets of size {}:\n{}'.format(size+1, frequent_itemset),end='\n\n')

Frequent itemsets of size 1:
['1', '4', '6', '3', '5', '8', '2', '7']

Frequent itemsets of size 2:
[('1', '4'), ('6', '3'), ('6', '5'), ('3', '5'), ('6', '8'), ('3', '8'), ('5', '8'), ('4', '3'), ('6', '2'), ('6', '7'), ('8', '2'), ('8', '7'), ('2', '7')]

Frequent itemsets of size 3:
[('6', '3', '5'), ('6', '3', '8'), ('6', '5', '8'), ('3', '5', '8'), ('6', '8', '2'), ('6', '8', '7'), ('6', '2', '7'), ('8', '2', '7')]

Frequent itemsets of size 4:
[('6', '3', '5', '8'), ('6', '8', '2', '7')]



#### Limitations
It is important to notice that the PCY algorithm is an extension to the Apriori algorithm. It partially addresses one of the issues of the Apriori algorithm, more specifically the heavy workload necessary to find frequent itemsets of size 2. This bottleneck of the Apriori algorithm is minimized by cleverly using a hashing technique. This technique provide a new filter that is used at the end of the first pass to reduce the size of the set of candidate pairs that will have to be checked during the second pass.<br>
Note however that the efficiency of this technique depends heavily on the hash function. In the extreme case where all pairs are hashed to the same bucket, the PCY extension does not provide any help since it won't filter out any candidates.<br>
This PCY algorithm does not propose any solution to the fact that we still have to go through the database multiple times to find frequent itemsets of increasing size.

### Other approaches
Let's now turn our attention to other algorithms/extensions. We will briefly introduce them and refer you to other sources to get more information.
#### Multihash algorithm
In the continuity of the PCY algorithm, this one proposes to hash, during the first pass, all pairs but **multiple times with different hash functions**. This is done in the hope that even if two different unfrequent pairs hash to the same bucket, with the first hash function, such that they both have to be kept as candidates because the bucket's "count' $\geq$ support threshold, the second hash function will hash them to different buckets, thus increasing the chance that they will be filtered out. One might then want to use many hash functions but due to the limited size of the memory, more hash functions means that each one will have a restricted space. Thus the more hash functions you have, the less buckets each of them will be able to have, increasing then the chance of collisions. There is thus clearly a tradeoff to be made.<br><br>
Source to go further: __[Mining of Massive Datasets](http://www.mmds.org/)__
#### Toivonen's algorithm
This algorithm attempts to limit the number of passes needed to find frequent items. In the best case scenario, this algorithm will only need two passes through the database to find all frequent itemsets (of any size $s$). However, this comes with a cost: the algorithm is not guaranteed to return a result at each run. Hence it might be necessary to rerun it multiple time. During it's first pass, Toivonen's algorithm will use a sample of the database and find all frequent itemsets that are in it, according to a lowered support threshold (set according to the sample's size). Next to those frequent itemsets, the algorithm also constructs a list called **Negative Border** which contains all itemsets not deemed frequent in the sample but for which all its immediate subsets are. During the second pass, the algorithm uses the full database and checks which of the frequent itemsets from the sample are also frequent in the database. Moreover, during this last pass, it checks that none of the itemsets from the Negative Border is frequent in the database. If this is the case, then the algorithm has to be rerun because it might be that one frequent itemsets has not been discovered. In the other case, the algorithm returns the list of frequent itemsets.<br><br>
Source to go further: __[Mining of Massive Datasets](http://www.mmds.org/)__
#### FP-Tree
This algorithm tries to store the frequent itemsets more efficiently. The retrieval of those is also reimagined. I made a python implementation of this interesting algorithm. You can check it here: **SOURCE**

### Conclusion
We have seen multiple approaches that can be used to mine frequent itemsets from a database of transactions.<br>
The extensions that we have covered, such as the hashing technique, and the sampling of the database, are neat tricks that can often be used in other context and scenario. This is why it is often useful to keep in mind tricks that helped speed up algorithms in other areas because they can often be applied to other algorithms in other circumstances. 

***